In [ ]:
!pip install termcolor
!pip install pandas_ta

In [1]:
from kiteconnect import KiteConnect
import urllib.parse
from requests import auth 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os, time
from os.path import expanduser
from selenium.webdriver.chrome.options import Options
import pyotp
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
import pdb

import sys
import hmac, base64, struct, hashlib, time
import requests


zerodha_cred = {
    'User':{
        'user_id' : 'RK2267',
        'password': "Shan@#2021",
        'api_key' : "pv2830q1vbrhu1eu",   
        'common_ans' : "",
        'api_secret' :'gz426yatawim9xruj2p6e51c40k44r7n',  
        'auth_key': 'TOOAIR75STSUWM67LS3NVUX7B2LGKJ7R'
    },

}
my_api_key=zerodha_cred['User']['api_key']

'''def telegram_bot_sendtext(my_message='SYSTEM RUNNING \n ------------------------- \nNo input received from program'):
    bot_token = '2020880061:AAHDR4kpcb4Y7lsbq59uUpgcqq7hiHYK_AE'  # ParkingBot ID
    bot_chatID = '-746818656'  # TelegramGroup ID 746818656

    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + my_message
    response = requests.get(send_text)
    return response.json()

telegram_bot_sendtext("Starting Zerodha Auth Key fetching")'''


def get_hotp_token(secret, intervals_no):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", intervals_no)
    h = hmac.new(key, msg, hashlib.sha1).digest()
    o = h[19] & 15
    h = (struct.unpack(">I", h[o:o+4])[0] & 0x7fffffff) % 1000000
    return h

def get_totp_token(secret):
    totp = str(get_hotp_token(secret, intervals_no=int(time.time())//30))
    if len(totp) < 6:
        totp = '0' + totp
    return totp


def getkite(_client):

    user_id = zerodha_cred[_client]['user_id']
    password = zerodha_cred[_client]['password']
    api_key = zerodha_cred[_client]['api_key']
    kite = KiteConnect(api_key=api_key)
    # common_ans = zerodha_cred[_client]['common_ans']
    auth_key = zerodha_cred[_client]['auth_key']
    url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
    # telegram_bot_sendtext("Line 92")

    try:
        options = Options()
        options.add_argument('--no-sandbox')
        options.add_argument('--headless')
        options.add_argument('--disble-dev-shm-usage')
        driver = webdriver.Chrome(ChromeDriverManager().install())
        driver.get(url)
        time.sleep(2)
        input_fields = driver.find_elements_by_tag_name('input')
        input_fields[0].send_keys(user_id)
        input_fields[1].send_keys(password)
        input_fields[1].send_keys(Keys.ENTER)
        time.sleep(1)
        two_fa = driver.find_elements_by_tag_name('input')
        common_ans = get_totp_token(auth_key)
        two_fa[0].send_keys(str(common_ans))
        two_fa[0].send_keys(Keys.ENTER)
        time.sleep(1)
        while("request_token=" not in driver.current_url):
            print (driver.current_url)
            driver.save_screenshot('login.png')
            time.sleep(3)
        redirect_url = driver.current_url
        #print('redirectURL-',redirect_url)
        parsed = urllib.parse.urlparse(redirect_url)
        query_dict = dict(urllib.parse.parse_qsl(parsed.query))
        request_token = query_dict['request_token']
        driver.close
        return request_token
    except Exception as e:
        print('closing')
        driver.close()
def gen_acc_token():
    global acc_key
    users = ['User'] 
    auth_keys={}
    cur_date=datetime.now().strftime("%y_%m_%d")

    for _client in users:
        api_key = zerodha_cred[_client]['api_key']
        api_secret = zerodha_cred[_client]['api_secret']

        req_token=getkite(_client)  
        # req_token = '8DoJtcPaDkOEj0bxCOm4KC7u4Y3QUd8H'
        url  = 'https://kite.trade/connect/login?v=3&api_key='+api_key 
        print ('url-',url)
        # pdb.set_trace()
        # telegram_bot_sendtext("Line 186")
        # pdb.set_trace()
        kite = KiteConnect(api_key=api_key)
        data = kite.generate_session(req_token, api_secret=api_secret)
        access_token=data["access_token"]
        print(data['user_name'],'has successfully signed in.')
        acc_key=access_token
        print('Access_token->',access_token)
        auth_keys[_client] = access_token

        auth_keys['date'] = cur_date
        telegram_bot_sendtext("*Auth keys extracted!*\n\nUser = \t\t" + _client   +  "\nAuth Key = \t" + auth_keys[_client] +  "\nDate = \t" + auth_keys['date'])

if __name__ == '__main__':
    try:
        gen_acc_token()
        acc_key=""
    except Exception as e:
        print('Happy Trading !')
        #print(my_api_key,acc_key)
        #telegram_bot_sendtext(f" Error in Token Generation - {str(e)}")

url- https://kite.trade/connect/login?v=3&api_key=pv2830q1vbrhu1eu
Kakal Krishna Rao has successfully signed in.
Access_token-> E43WfLZn1AfXJ54vVXHndfxRIZOg276d
Happy Trading !


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))

In [3]:
from datetime import *

def history_data_cpr(ticker_val):
    data1=['' for i in range(1)]
    start_date=(datetime.now()-timedelta(days=1029))
    new_date=(datetime.now())
    
    startd=pd.to_datetime(str(start_date)[:10])
    endd=pd.to_datetime(str(new_date))
    #print(startd,endd)
    
    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/day?from="+str(startd)+"&to="+str(endd)
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [4]:
from datetime import datetime, timedelta
data={}

def cpr_day(df):
    i=0
    enter=False
    k=1
    m=0
    global data
    data={0:{0:'1',1:'1',2:'1'}}
    x=0
    year=2020
    while(i<len(df)):
        
        if pd.to_datetime(df.Time.iloc[i]).year==year+1:
            #print('NEW-YEAR')
            k=1
            m=0
            year=year+1
            enter=False
           
        
        if pd.to_datetime(df.Time.iloc[i]).month==k and not enter:
            start=pd.to_datetime(df['Time'].iloc[i])
            m=k+1
            enter=True
            low=float('inf')
            high=float('-inf')
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            #print('Enter',df.Time.iloc[i],m,k)
            
        if enter and pd.to_datetime(df.Time.iloc[i]).month!=m:
            #print('on',df.Time.iloc[i])
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            
        if enter and df.Time.iloc[i]==df.Time.iloc[-1]:
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            close=df['Close'].iloc[i]
            pivot=(high+low+close)/3
            bc=(high+low)/2
            tc=(pivot-bc)+pivot
            dt=str(start)
            st=dt[:10]+'T'+'00:00:00+0530'
            km={'Start':st,'End':df['Time'].iloc[i],'Pivot':pivot,'BC':bc,'TC':tc}
            #print(km)
            data[x]=km
            x+=1
            
            
        if enter and pd.to_datetime(df.Time.iloc[i]).month==m:
            k+=1
            m+=1
            i-=1
            enter=False
            #print('final',df.Time.iloc[i])
            low=min(low,df['Low'].iloc[i])
            high=max(high,df['High'].iloc[i])
            close=df['Close'].iloc[i]
            pivot=(high+low+close)/3
            bc=(high+low)/2
            tc=(pivot-bc)+pivot
            dt=str(start)
            st=dt[:10]+'T'+'00:00:00+0530'
            km={'Start':st,'End':df['Time'].iloc[i],'Pivot':pivot,'BC':bc,'TC':tc}
            #print(km)
            data[x]=km
            x+=1
            
            
        i+=1

In [5]:
#%%capture cap --no-stderr

from termcolor import colored
import pandas_ta as ta
import pandas as pd
from datetime import *

cpr_1day={}
cpr_1day_dict={}

def crossover(dfx,tkname,dx):
   
    dfx['Pivot']=0.0
    dfx['BC']=0.0
    dfx['TC']=0.0
    enter=False
    k=1
    x=""
    for i in range(1,len(dfx)):
        
        if k==len(data):
            break
        x=dx[k]['Start']
        
        if str(dfx['Time'].iloc[i])==x:
            #print('Enter',x)
            enter=True
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
        if enter and dfx['Time'].iloc[i]==dx[k]['End']:
            #print('last',dfx['Time'].iloc[i])
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
            #print(dfx['Pivot'].iloc[i],dfx['BC'].iloc[i])
            k+=1
            enter=False
        elif enter and dfx['Time'].iloc[i]!=dx[k]['End']:
            x=""
            #print('wk',dfx['Time'].iloc[i])
            dfx['Pivot'].iloc[i]=dx[k-1]['Pivot']
            dfx['BC'].iloc[i]=dx[k-1]['BC']
            dfx['TC'].iloc[i]=dx[k-1]['TC']
            #print(dfx['Pivot'].iloc[i],dfx['BC'].iloc[i])
    
    #cpr_1day[tkname]=df
    k={}
    k['Close']=dfx['Close'].iloc[-2]
    k['Pivot']=dfx['Pivot'].iloc[-2]
    k['BC']=dfx['BC'].iloc[-2]
    k['TC']=dfx['TC'].iloc[-2]
    
    cpr_1day_dict[tkname]=k
            

    
    
       
    #print('------------------------------------------------------------------------------------------------------------')
    
    
cpr_data={}
#dicct={4267265: 'BAJAJ-AUTO'}
dicct={5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 60417: 'ASIANPAINT', 1510401: 'AXISBANK', 4268801: 'BAJAJFINSV', 1195009: 'BANKBARODA', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 5105409: 'DEEPAKNTR', 3771393: 'DLF', 225537: 'DRREDDY', 232961: 'EICHERMOT', 261889: 'FEDERALBNK', 589569: 'HAL', 1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 345089: 'HEROMOTOCO', 348929: 'HINDALCO', 356865: 'HINDUNILVR', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB', 387073: 'INDHOTEL', 1346049: 'INDUSINDBK', 408065: 'INFY', 3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 492033: 'KOTAKBANK', 2939649: 'LT', 519937: 'M&M', 2815745: 'MARUTI', 2674433: 'MCDOWELL-N', 3675137: 'MINDTREE', 2730497: 'PNB', 3834113: 'POWERGRID', 4708097: 'RBLBANK', 738561: 'RELIANCE', 779521: 'SBIN', 857857: 'SUNPHARMA', 871681: 'TATACHEM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL', 2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 2170625: 'TVSMOTOR', 969473: 'WIPRO', 975873: 'ZEEL'}
tokens=list(dicct.keys())
for tk in tokens:
    df=history_data_cpr(tk)[0]
    cpr_day(df)
    cpr_data[tk]=data[list(data.keys())[-2]]
    crossover(df,dicct[tk],data)

In [ ]:
cpr_1day_dict

In [6]:
from datetime import *
def history_data(ticker_val):
    data1=['' for i in range(1)]
    t2=(datetime.now() - timedelta(days=0))
    t1=(datetime.now() - timedelta(days=1))
    curr_hr=str(t2).split(':')[0].split(' ')[-1]
    curr_min=str(t2).split(':')[1]
    curr_secs=str(t2).split(':')[2]
    final=curr_hr+':'+curr_min+':'+curr_secs

    t1=str(t1)
    t2=str(t2)
    t1=t1.split(" ")
    t3=t1[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t1=t1[0]+"+"+t3
    t2=t2.split(" ")
    t3=t2[1].split(':')
    t3[0]='9'
    t3[1]='00'
    t3=t3[0]+':'+t3[1]+':'+t3[2]
    t2=t2[0]+"+"+final
    t1,t2

    url1="https://api.kite.trade/instruments/historical/"+str(ticker_val)+"/minute?from="+t1+"&to="+t2
    #print(url1)
    HEADERS = {"X-Kite-Version": "3", "Authorization":"token pv2830q1vbrhu1eu:"+acc_key}
    res1 = requests.get(url1, headers=HEADERS)
    i=0
    data1[i] = res1.json()
    data1[i] = data1[i]["data"]["candles"]
    data1[i]=pd.DataFrame(data1[i])
    data1[i] = data1[i].rename(columns={0: 'Time', 1: 'Open', 2:'High', 3:'Low', 4:'Close', 5:'Volume'})
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    #print('History_data {} collected .'.format(ticker_val))
    return data1

In [13]:
import threading
import logging
from kiteconnect import KiteTicker
from termcolor import colored
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
from multiprocessing.pool import ThreadPool

rex=set()
mx=set()
st=set()
kws = KiteTicker(my_api_key, acc_key)
#dicct={4267265: 'BAJAJ-AUTO'}
dicct={5633: 'ACC', 6401: 'ADANIENT', 3861249: 'ADANIPORTS', 325121: 'AMBUJACEM', 60417: 'ASIANPAINT', 1510401: 'AXISBANK', 4268801: 'BAJAJFINSV', 1195009: 'BANKBARODA', 108033: 'BHARATFORG', 2714625: 'BHARTIARTL', 112129: 'BHEL', 2763265: 'CANBK', 177665: 'CIPLA', 5215745: 'COALINDIA', 5105409: 'DEEPAKNTR', 3771393: 'DLF', 225537: 'DRREDDY', 232961: 'EICHERMOT', 261889: 'FEDERALBNK', 589569: 'HAL', 1850625: 'HCLTECH', 340481: 'HDFC', 341249: 'HDFCBANK', 345089: 'HEROMOTOCO', 348929: 'HINDALCO', 356865: 'HINDUNILVR', 1270529: 'ICICIBANK', 2863105: 'IDFCFIRSTB', 387073: 'INDHOTEL', 1346049: 'INDUSINDBK', 408065: 'INFY', 3484417: 'IRCTC', 424961: 'ITC', 1723649: 'JINDALSTEL', 3001089: 'JSWSTEEL', 492033: 'KOTAKBANK', 2939649: 'LT', 519937: 'M&M', 2815745: 'MARUTI', 2674433: 'MCDOWELL-N', 3675137: 'MINDTREE', 2730497: 'PNB', 3834113: 'POWERGRID', 4708097: 'RBLBANK', 738561: 'RELIANCE', 779521: 'SBIN', 857857: 'SUNPHARMA', 871681: 'TATACHEM', 884737: 'TATAMOTORS', 877057: 'TATAPOWER', 895745: 'TATASTEEL', 2953217: 'TCS', 3465729: 'TECHM', 897537: 'TITAN', 2170625: 'TVSMOTOR', 969473: 'WIPRO', 975873: 'ZEEL'}
tokens=list(dicct.keys())

url='https://api.telegram.org/bot5761331728:AAEeP9GPp69JUYXwtfw5ysP8HwugoHXZdGw/sendMessage?chat_id=-665312475&text='
'''tokens=[60750087,59897095,60128519]
dicct={60750087:'SILVERMIC',59897095:'SILVERM',60128519:'SILVER'}'''
flag=False
x=-1
x2=-1
s=""
count=0

for i in tokens:
    prev=history_data(i)[0]
    globals()['prevclose'+'_{}'.format(i)]=prev['Close'].iloc[-1]
    
    
            
def calculate(tkname,s,ltp,ohlc):
    global rex,x,dicct,tokens,mx,flag,st,cpr_1day_dict
    cpx=cpr_1day_dict[dicct[tkname]]
    #print(tx)
    if x^(s.minute)==0:
        if (s.second==59 or s.second==58) and tkname not in st:
            st.add(tkname)
            res={}
            res={'Ticker':dicct[tkname],'Time':s,'Close':ltp,'Pivot':cpx['Pivot'],'prev':globals()['prevclose'+'_{}'.format(tkname)]}
            print(res)
            if globals()['prevclose'+'_{}'.format(tkname)] < max(cpx['TC'],cpx['BC']) and ltp>max(cpx['TC'],cpx['BC']):
                print('B Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)
                requests.get(url+'DAILY-> BUY {} at {}'.format(dicct[tkname],ltp),stream=True)

            elif ltp<min(cpx['TC'],cpx['BC']) and globals()['prevclose'+'_{}'.format(tkname)]>max(cpx['TC'],cpx['BC']):
                requests.get(url+'DAILY-> SELL {} at {}'.format(dicct[tkname],ltp),stream=True)
                print('S Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)

            else:
                print('N Prev:',globals()['prevclose'+'_{}'.format(tkname)],'max',max(cpx['TC'],cpx['BC']),'Close',ltp)


            globals()['prevclose'+'_{}'.format(tkname)]=ltp
            print()

    else:
        x=s.minute
        #globals()['prevclose'+'_{}'.format(tkname)]=ltp
        flag=False
        st=set()
        print("-------------------------START OF {} mins-----------------------".format(x))
    
    

            
def on_ticks(ws, ticks):
    logging.debug("Ticks: {}".format(ticks))
    tim=-1
    thread_list=[]
    for turn in range(len(ticks)):
        ltp=ticks[turn]['last_price']
        ohlc=ticks[turn]['ohlc']
        T2=datetime.now()
        hr2=int(T2.hour)
        min2=int(T2.minute)
        sec2=int(T2.second)
        tk=ticks[turn]['instrument_token']
        #T4=str(T2).split(' ')[0]
        TR1=str(T2).split(' ')[1]
        s=pd.to_datetime(TR1)
        try:
            t1=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            #t2=threading.Thread(target=calculate,args=[tk,s,ltp,ohlc])
            t1.start()
            t1.join()
        except KeyError:
            pass
        
        
def on_connect(ws, response):
    print("Trading starting soon....")
    check_flag=False
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, tokens)
    print("Daily Trading Started. Good Luck....")
    requests.get(url+'Daily Trading Started',stream=True)
          
            
def on_close(ws, code, reason):

    # kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)
    # print("reconnecting...")
    requests.get(url+'Closing Connection',stream=True)
    print("closing connection")
    logging.info("Connection closed: {code} - {reason}".format(code=code, reason=reason))
    # ws.stop()

def on_error(ws, code, reason):
    logging.info("123456Connection error: {code} - {reason}".format(code=code, reason=reason))


def on_reconnect(ws, attempts_count):
    logging.info("123456Reconnecting: {}".format(attempts_count))


def on_noreconnect(ws):
    logging.info("123456Reconnect failed.")


# Assign the callbacks.
kws.on_ticks = on_ticks

kws.on_close = on_close
kws.on_error = on_error
kws.on_connect = on_connect
kws.on_reconnect = on_reconnect
kws.on_noreconnect = on_noreconnect
kws.connect(threaded=True, disable_ssl_verification=False)


-------------------------START OF 23 mins-----------------------
Trading starting soon....
Daily Trading Started. Good Luck....
{'Ticker': 'EICHERMOT', 'Time': Timestamp('2022-11-01 15:23:58.333797'), 'Close': 3801.05, 'Pivot': 3566.616666666667, 'prev': 3803.0}
N Prev: 3803.0 max 3619.1083333333336 Close 3801.05

{'Ticker': 'SUNPHARMA', 'Time': Timestamp('2022-11-01 15:23:58.337162'), 'Close': 1037.1, 'Pivot': 920.3166666666666, 'prev': 1037.0}
N Prev: 1037.0 max 934.4833333333332 Close 1037.1

{'Ticker': 'AXISBANK', 'Time': Timestamp('2022-11-01 15:23:58.338325'), 'Close': 873.25, 'Pivot': 752.3666666666668, 'prev': 873.1}
N Prev: 873.1 max 761.95 Close 873.25

{'Ticker': 'M&M', 'Time': Timestamp('2022-11-01 15:23:58.339113'), 'Close': 1353.85, 'Pivot': 1272.4833333333333, 'prev': 1354.3}
N Prev: 1354.3 max 1274.625 Close 1353.85

{'Ticker': 'MINDTREE', 'Time': Timestamp('2022-11-01 15:23:58.339846'), 'Close': 3515.0, 'Pivot': 3191.7166666666667, 'prev': 3517.0}
N Prev: 3517.0 max 32